Wczytanie zbioru danych

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import pandas as pd

file_path = '/content/drive/My Drive/data_sety/ready_data_set.csv'
data_set = pd.read_csv(file_path)
print(data_set.head())

   label                                               text  \
0      1  ounce feather bowl hummingbird opec moment ala...   
1      1  wulvob get your medircations online qnb ikud v...   
2      0   computer connection from cnn com wednesday es...   
3      1  university degree obtain a prosperous future m...   
4      0  thanks for all your answers guys i know i shou...   

                                       filtered_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  thanks for all your answers guys i know i shou...   

                                     normalized_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  t

Zamiana ciągów znaków na listę słów

In [3]:
from gensim.models import Word2Vec

import ast
data_set['lemmatized_text'] = data_set['lemmatized_text'].apply(ast.literal_eval)

In [4]:
print(data_set.head())

   label                                               text  \
0      1  ounce feather bowl hummingbird opec moment ala...   
1      1  wulvob get your medircations online qnb ikud v...   
2      0   computer connection from cnn com wednesday es...   
3      1  university degree obtain a prosperous future m...   
4      0  thanks for all your answers guys i know i shou...   

                                       filtered_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  thanks for all your answers guys i know i shou...   

                                     normalized_text  \
0  ounce feather bowl hummingbird opec moment ala...   
1  wulvob get your medircations online qnb ikud v...   
2   computer connection from cnn com wednesday es...   
3  university degree obtain a prosperous future m...   
4  t

Trenowanie modelu word2vec. sentences - dane do treningu, vector_size - rozmiar wektora cech (wymiarów) dla każdego słowa, window - maksymalna odległość między aktualnym słowem a przewidywanym w kontekście, min_count - minimalna liczba wystąpień słowa w dokumencie, aby było ono uwzględnione w treningu, workers - liczba wątków roboczych do trenowania modelu

In [5]:
model = Word2Vec(sentences=data_set['lemmatized_text'], vector_size=100, window=5, min_count=1, workers=4)

Zapisanie modelu w folderze

In [6]:
from gensim.models import Word2Vec

model_path = '/content/drive/My Drive/data_sety/word_models/word2vec_model.model'
model.save(model_path)

Osadzanie tekstów w całym zbiorze danych

In [7]:
import numpy as np

def document_word2vec_embedding(doc, model):
    vectors = [model.wv[word] for word in doc if word in model.wv]

    if len(vectors) == 0:
        return np.zeros(model.vector_size)

    return np.mean(vectors, axis=0)

word2vec_embeddings = np.array([document_word2vec_embedding(doc, model) for doc in data_set['lemmatized_text']])


Normalizacja wektorów osadzeń do normy euklidesowej - Dzięki temu różnice w długości wektorów nie wpłyną na wyniki algorytmów klasteryzacji

Po normalizacji, punkty są skalowane w taki sposób, by ich odległość od początku układu współrzędnych wynosiła 1.

In [8]:
from sklearn.preprocessing import normalize

normalized_embeddings = normalize(word2vec_embeddings, norm='l2', axis=1)

word2vec_embeddings_df = pd.DataFrame(normalized_embeddings)

Zapisanie zbioru gotowego do klasteryzacji

In [9]:
word2vec_embeddings_df.to_csv('/content/drive/My Drive/data_sety/normalized_word2vec_embeddings.csv', index=False)